In [ ]:
# Please note that you may not have to run the command here, if you already have openai SDK installed
# Upgrade to 
# Python SDK v1.2
# with 
%pip install --upgrade openai

In [ ]:
# Load .env file

# Import load_dotenv
from dotenv import load_dotenv
import os
import openai

load_dotenv()

api_key = os.environ.get("OPENAI_API_KEY")
print(api_key)
openai.api_key = api_key#os.environ["OPENAI_API_KEY"]  # Set your OpenAI API key
print (openai.api_key)

In [2]:
# new
from openai import OpenAI

client = OpenAI(
  api_key = os.environ.get("OPENAI_API_KEY")
  # api_key=os.environ['OPENAI_API_KEY'],  # this is also the default, it can be omitted
)

## Step 1: Create an Assistant
An Assistant represents an entity that can be configured to respond to users’ Messages using several parameters like:

Instructions: how the Assistant and model should behave or respond

Model: you can specify any GPT-3.5 or GPT-4 models, including fine-tuned models. 

Tools: the API supports Code Interpreter and Retrieval that are built and hosted by OpenAI.

Functions: the API allows you to define custom function signatures, with similar behavior as our function calling feature.

In this example, we're following the [Assistants API Overview (Python SDK)](https://cookbook.openai.com/examples/assistants_api_overview_python):

In [3]:
assistant = client.beta.assistants.create(
    name="Math Tutor",
    instructions="You are a personal math tutor. Write and run code to answer math questions.",
    tools=[{"type": "code_interpreter"}],
    model="gpt-4o-mini",
)

### Step 2: Create a Thread

A Thread represents a conversation. We recommend creating one Thread per user as soon as the user initiates the conversation. Pass any user-specific context and files in this thread by creating Messages.

In [4]:
thread = client.beta.threads.create()

### Step 3: Add a Message to a Thread
A Message contains text, and optionally any files that you allow the user to upload. Messages need to be added to a specific Thread. Adding images via message objects like in Chat Completions using GPT-4 with Vision is not supported today, but we plan to add support for them in the coming months. You can still upload images and have them processes via retrieval.

In [5]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I need to solve the equation `3x + 11 = 14`. Can you help me?"
)

In [ ]:
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions="Please address the user as Jane Doe. The user has a premium account.",
)

print("Run completed with status: " + run.status)

if run.status == "completed":
    messages = client.beta.threads.messages.list(thread_id=thread.id)

    print("messages: ")
    for message in messages:
        assert message.content[0].type == "text"
        print({"role": message.role, "message": message.content[0].text.value})

    client.beta.assistants.delete(assistant.id)